Using [This link](https://github.com/cloudtostreet/MODIS_GlobalFloodDatabase).

Tellman et al, Satellite imaging reveals increased proportion of population exposed to floods; Nature; https://doi.org/10.1038/s41586-021-03695-w

Idai: 4725

main_floodmechanism.txt <br>
main_gfd.py

In [3]:
# initialize GEE
import ee
from os.path import join


# try:
#     ee.Initialize()
# except:
#     service_account = "hybrid-models@hybridmodels-354115.iam.gserviceaccount.com"
#     credentials = ee.ServiceAccountCredentials(service_account, join("gcloud_keys", ".hybridmodels-354115-e71f122c7f06.json"))
#     ee.Initialize(credentials)
    
!earthengine authenticate
            
%cd "/Users/alison/Documents/DPhil/hybridmodels/MODIS_GlobalFloodDatabase"

/Users/alison/Documents/DPhil/hybridmodels/MODIS_GlobalFloodDatabase


In [13]:
from flood_detection import modis
from flood_detection.utils import export, misc

import time, os, csv

In [16]:
gfd = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')
gfdFloodedSum = gfd.select('flooded').sum()  # all satellite-derived global floodplains

In [151]:
import pandas as pd

keywords = ['tropical', 'cyclone', 'storm', 'typhoon', 'hurricane', 'surge']
keywords = keywords + [keyword.capitalize() for keyword in keywords]

events_table = pd.read_csv('gfd_event_stats_20215_13_error_fixed_2.csv')
events_table = events_table[events_table['dfo_cause'].apply(lambda x: any(map(x.lower().__contains__, keywords)))]
events_table['dfo_began'] = pd.to_datetime(events_table['dfo_began'])
events_table = events_table[events_table['dfo_began'] >= pd.to_datetime('2017-01-01')]

In [152]:
cols_of_interest = ['system:index', 'threshold_type','dfo_country',
                    'dfo_other_country', 'dfo_centroid_x', 'dfo_centroid_y',
                    'dfo_began', 'dfo_ended', 'dfo_validation',
                    'dfo_dead', 'dfo_displaced', 'dfo_cause', 
                    'dfo_severity', 'gfd_area']

In [153]:
# manual
index_dict = {4516: 'Hurricane Irma',
             4676: 'Tropical Storm Florence',
             4695: 'Hurricane Willa'}

events_table = events_table.set_index('index')
events_table[cols_of_interest]

,system:index,threshold_type,dfo_country,dfo_other_country,dfo_centroid_x,dfo_centroid_y,dfo_began,dfo_ended,dfo_validation,dfo_dead,dfo_displaced,dfo_cause,dfo_severity,gfd_area
index,,,,,,,,,,,,,,
4458,DFO_4458_From_20170328_to_20170408,otsu,Australia,New Zealand,148.681590,-21.974973,2017-03-28,4/8/17,News,6,20000,"Tropical Storm, Surge",2.0,3898.29
4462,DFO_4462_From_20170405_to_20170421,std,Argentina,NaN,-67.938778,-45.953281,2017-04-05,4/21/17,News,1,8000,"Tropical Storm, Surge",1.5,1418.57
4515,DFO_4515_From_20170823_to_20170826,std,China,NaN,114.138854,22.385119,2017-08-23,8/26/17,News,16,0,"Tropical Storm, Surge",2.0,202.12
4516,DFO_4516_From_20170908_to_20170919,std,USA,Cuba,-81.625156,28.594245,2017-09-08,9/19/17,News,84,3000,"Tropical Storm, Surge",2.0,9990.79
4590,DFO_4590_From_20180316_to_20180323,std,Madagascar,NaN,47.827095,-16.669172,2018-03-16,3/23/18,FloodList,17,6000,"Tropical Storm, Surge",1.5,994.21
4652,DFO_4652_From_20180717_to_20180723,std,Cambodia,NaN,104.477000,11.988800,2018-07-17,7/23/18,FloodList,0,6800,"Tropical Storm, Surge",1.5,742.49
4653,DFO_4653_From_20180717_to_20180723,otsu,Vietnam,NaN,105.925000,21.173600,2018-07-17,7/23/18,FloodList,5,1400,"Tropical Storm, Surge",1.5,1850.43
4676,DFO_4676_From_20180915_to_20181002,std,USA,NaN,-78.909100,35.064200,2018-09-15,10/2/18,FloodList,12,1500000,"Tropical Storm, Surge",2.0,437.99
4695,DFO_4695_From_20181023_to_20181027,std,Mexico,NaN,-105.236000,23.721800,2018-10-23,10/27/18,News Reports,0,0,"Tropical Storm, Surge",1.5,2720.37


In [197]:
stormDartmouthId = 4695
storms = ee.ImageCollection(
    gfd.filterMetadata('id', 'equals', stormDartmouthId))

print(f"Number of flood maps: {storms.size().getInfo()}")
storm = ee.Image(storms.first())

# extract metadata
start = storm.get('began').getInfo()
end = storm.get('ended').getInfo()
cause = storm.get('dfo_main_cause').getInfo()
severity = storm.get('dfo_severity').getInfo()
country = storm.get('dfo_country').getInfo()

print(f"start date: {start}")
print(f"end date: {end}")
print(f"cause: {cause}")
print(f"severity: {severity}")
print(f"main country: {country}")

lat = storm.get('dfo_centroid_x').getInfo()
lon = storm.get('dfo_centroid_y').getInfo()

Number of flood maps: 1
start date: 2018-10-23
end date: 2018-10-27
cause: Tropical Storm Willa
severity: 1.5
main country: Mexico


In [211]:
# set up footprint
footprint = storm.get('system:footprint').getInfo()
footprint = ee.Geometry.LinearRing(footprint['coordinates'])

The main issue here is that we don't have a specific date. I am unsure how to resolve this to be compatible with the current form of the data.

In [243]:
band = 'duration'

task_config = {
    'scale': 30,  
    'region': footprint,
    'maxPixels': 5000000000
    }

task = ee.batch.Export.image(storm.select(band), f"{index_dict[stormDartmouthId]} - {band}", task_config)

task.start()

In [249]:
task.status()

{'state': 'RUNNING',
 'description': 'Hurricane Willa - duration',
 'creation_timestamp_ms': 1678907908827,
 'update_timestamp_ms': 1678908767503,
 'start_timestamp_ms': 1678907927061,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'MUEBNHTQ47CNEOBRPIKB2KV5',
 'name': 'projects/earthengine-legacy/operations/MUEBNHTQ47CNEOBRPIKB2KV5'}

In [246]:
band = 'jrc_permwater'

task_config = {
    'scale': 30,  
    'region': footprint,
    'maxPixels': 5000000000
    }

task1 = ee.batch.Export.image(storm.select(band), f"{index_dict[stormDartmouthId]} - {band}", task_config)

task1.start()

In [247]:
band = 'clear_perc'

task_config = {
    'scale': 30,  
    'region': footprint,
    'maxPixels': 5000000000
    }

task2 = ee.batch.Export.image(storm.select(band), f"{index_dict[stormDartmouthId]} - {band}", task_config)

task2.start()

In [193]:
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [198]:
durationPalette = ['C3EFFE', '1341E8', '051CB0', '001133']

m = folium.Map(location=[lon, lat], zoom_start=5)

# map extent
vizParams = {'min': 0, 'max': 1, 'palette': '001133'}
m.add_ee_layer(storm.select('flooded').selfMask(), vizParams, f"index_dict[stormDartmouthId] - Inundation Extent")

# durations
durationPalette = ['C3EFFE', '1341E8', '051CB0', '001133'];
m.add_ee_layer(storm.select('duration').selfMask(), {'min': 0, 'max': 4, 'palette': durationPalette},
  f"index_dict[stormDartmouthId] - Duration")

# all floods to view satellite-derived flood plain
m.add_ee_layer(gfdFloodedSum.selfMask(), {'min': 0, 'max': 10, 'palette': durationPalette}, 'GFD Satellite Observed Flood Plain');

# 
jrc = gfd.select('jrc_perm_water').sum().gte(1)
m.add_ee_layer(jrc.selfMask(), {'min': 0, 'max': 1, 'palette': 'C3EFFE'}, 'JRC Permanent Water')

m